In [12]:
import streamlit as st
import re


def parse_scale_label(label):
    """
    체크박스 라벨 (ex: "매우 불만족(1)")에서 숫자(int)만 추출하여 반환.
    - label: "매우 만족(5)"와 같이 괄호 안에 숫자가 들어있는 문자열
    - return: 5 (int), 만약 찾지 못하면 None
    """
    match = re.search(r"\((\d+)\)", label)
    if match:
        return int(match.group(1))
    else:
        return None

In [13]:
def _on_change_checkbox_single(val, scale_values, key_prefix):
    """
    체크박스가 클릭될 때(ON/OFF) 호출되는 콜백.
    """
    is_checked = st.session_state[f"{key_prefix}_{val}"]
    if is_checked:
        # 새로 체크된 항목을 session_state에 저장
        st.session_state[f"{key_prefix}_selected"] = val
        # 다른 항목은 해제
        for v in scale_values:
            if v != val:
                st.session_state[f"{key_prefix}_{v}"] = False
    else:
        # 체크 해제됐을 때
        any_other_checked = False
        for v in scale_values:
            if st.session_state.get(f"{key_prefix}_{v}", False):
                st.session_state[f"{key_prefix}_selected"] = v
                any_other_checked = True
                break
        if not any_other_checked:
            st.session_state[f"{key_prefix}_selected"] = None

In [14]:
def checkbox_scale_single(question_text, scale_values, key_prefix):
    """
    [단일 선택] 체크박스 컴포넌트.
    - question_text: 문항 질문 텍스트
    - scale_values: 표시할 체크박스 라벨 목록 (예: ["매우 불만족(1)", ...])
    - key_prefix: 세션 키 prefix

    반환값: 사용자가 체크한 라벨(문자열) 또는 None
    """
    if f"{key_prefix}_selected" not in st.session_state:
        st.session_state[f"{key_prefix}_selected"] = None

    # 레이아웃: 왼쪽(질문), 오른쪽(체크박스들)
    col1, col2 = st.columns([3, 5])
    col1.write(question_text)

    # 체크박스들이 들어갈 열 (5항목 기준)
    list_cols = col2.columns([1, 1, 1, 1, 1])

    for idx, val in enumerate(scale_values):
        default_checked = st.session_state[f"{key_prefix}_selected"] == val
        list_cols[idx].checkbox(
            str(val),
            key=f"{key_prefix}_{val}",
            value=default_checked,
            on_change=_on_change_checkbox_single,
            args=(val, scale_values, key_prefix),
        )

    return st.session_state[f"{key_prefix}_selected"]

In [15]:
scale_labels = [
    "매우 불만족(1)",
    "불만족(2)",
    "보통(3)",
    "만족(4)",
    "매우 만족(5)",
]

# 문항별 그룹을 리스트로 정의
question_groups = [
    {
        "sub_title": "주행 중 느낀점 (승차감 중심)",
        "question_list": [
            ("Q1. 큰 도로나 고속도로 주행 시 안정감이 편안했나요?", "q1"),
            ("Q2. 커브길 주행 시 흔들림 없이 편하게 느끼셨나요?", "q2"),
            ("Q3. 좁은 길 주행 시 불안감 없이 안정적으로 느껴졌나요?", "q3"),
            ("Q4. 출발/멈출 때 급하게 흔들리지 않고 자연스러웠나요?", "q4"),
        ],
    },
    {
        "sub_title": "돌발 및 긴급 상황 대처 능력 평가",
        "question_list": [
            (
                "Q5. 갑자기 보행자나 자전거가 나타났을 때 빠르게 반응하고 잘 멈추거나 피했나요?",
                "q5",
            ),
            (
                "Q6. 예상 못한 앞차의 급정거 같은 긴급 상황에서 믿고 맡길 만큼 시스템이 잘 반응했나요?",
                "q6",
            ),
            ("Q7. 신호등이나 교통 표지판을 정확히 보고 잘 따르는 편인가요?", "q7"),
            (
                "Q8. 비가 오거나 밤에 어두울 때도 불안감 없이 안정적으로 주행했나요?",
                "q8",
            ),
        ],
    },
    {
        "sub_title": "자동 주차 기능 평가",
        "question_list": [
            ("Q9. 자동 주차할 때 주차 공간을 잘 찾아서 정확히 안내했나요?", "q9"),
            ("Q10. 자동으로 주차할 때 불안감 없이 편안하게 주차가 잘 되었나요?", "q10"),
            (
                "Q11. 주차 중 주변 차량이나 기둥 같은 장애물을 잘 파악하고 안전하게 주차했나요?",
                "q11",
            ),
            ("Q12. 주차가 끝난 후 차량이 주차선 안에 바르게 정렬되었나요?", "q12"),
        ],
    },
    {
        "sub_title": "전반적인 만족도 & 추천 의사",
        "question_list": [
            ("Q13. 전체적으로 자율주행 차량의 승차감이 마음에 들었나요?", "q13"),
            (
                "Q14. 자율주행 차량 조작방법이나 사용법이 쉽고 간단하게 느껴졌나요?",
                "q14",
            ),
            ("Q15. 앞으로도 이 자율주행 차량을 다시 이용하고 싶나요?", "q15"),
            ("Q16. 친구나 가족에게도 자율주행 차량을 추천할 생각이 있나요?", "q16"),
        ],
    },
]

In [16]:
responses = {}
for group in question_groups:

    for q in group["question_list"]:
        responses.update({q[1]: checkbox_scale_single(q[0], scale_labels, q[1])})

responses

2025-03-18 20:50:37.237 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 20:50:37.238 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 20:50:37.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 20:50:37.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 20:50:37.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 20:50:37.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 20:50:37.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-18 20:50:37.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

{'q1': None,
 'q2': None,
 'q3': None,
 'q4': None,
 'q5': None,
 'q6': None,
 'q7': None,
 'q8': None,
 'q9': None,
 'q10': None,
 'q11': None,
 'q12': None,
 'q13': None,
 'q14': None,
 'q15': None,
 'q16': None}